In [97]:
import fuzzywuzzy
from fuzzywuzzy import process, fuzz
import pandas as pd
merged = pd.read_csv('../Data/merged_title_movies_collection.csv')

In [98]:
merged

,Title,Language,Release date,Date First Available,Run time,Producers,Directors,Writers,Actors,Media Format,Subtitles,Genres
0,- Learn To Sing Gospel,[],"February 1, 2007",NaN,2 hours,NaN,"Greg Hannon, M.M.",[],"['Canagela Robertson', ' M.M.']",['NTSC'],[],NaN
1,- Learning To Flatpick,[],"June 3, 2003",NaN,4 hours,NaN,"Traum, Happy",[],"['Kaufman', ' Steve']","['Multiple Formats, Color, NTSC, Closed-captio...",[],NaN
2,100 Boating Mistakes & How To Avoid Them Trai...,[],"January 1, 2007",NaN,50 minutes,NaN,NaN,[],[],"['Multiple Formats, Color, NTSC']",[],NaN
3,100 Sailing Mistakes & How To Avoid Them Trai...,[],"January 1, 2007",NaN,50 minutes,James Marsh,NaN,[],[],"['Color, DVD, NTSC']",[],NaN
4,100 Saltwater Fishing Mistakes & How To Avoid...,[],"January 1, 2007",NaN,50 minutes,James Marsh,NaN,[],[],"['Multiple Formats, Color, NTSC']",[],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
152466,Águila o sol,['Spanish'],"June 30, 1996","September 29, 2006",NaN,NaN,Arcady Boytler,[],"['Cantinflas', ' Manuel Medel', ' Margarita Mo...",[],[],NaN
152467,Æon Flux,[],"June 3, 2008",NaN,1 hour and 33 minutes,NaN,Karyn Kusama,[],"['Charlize Theron', ' Frances McDormand', ' So...","['Multiple Formats, Blu-ray, Color, Dolby, NTS...","['English, French, Spanish']",NaN
152468,ÇA TWISTE À POPONGUINE (Rocking Poponguine),[],"September 10, 2009",NaN,NaN,NaN,Moussa Sene Absa,[],"['Coura Ba', ' Jean-François Balmer', ' Ousman...",['Available to watch on '],[],Arthouse
152469,Ça commence aujourd'hui,"['French (Dolby Digital 5.1), Italian (Dolby D...","January 21, 2003","June 27, 2006",1 hour and 57 minutes,"Alain Sarde, Frédéric Bourboulon",Bertrand Tavernier,[],"['Philippe Torreton', ' Maria Pitarresi', ' Na...",['PAL'],"['German, Dutch, English']",NaN


In [99]:
process.extract('K. Figura',['Katarzyna Figura'])

[('Katarzyna Figura', 86)]

In [100]:
def remove_values_from_list(the_list, val):
   return [value for value in the_list if value != val]

In [101]:
names = []

for index, row in merged.iterrows():
    # Actors
    single_actor_name = row['Actors'].strip("[]").split(', ')
    single_actor_name = [s.replace("'","").replace('"','').strip() for s in single_actor_name]
    for s in single_actor_name:
        if s != '' and s not in names:
            names.append(s)
    # Writers
    single_writer_name = row['Writers'].strip("[]").split(', ')
    single_writer_name = [s.replace("'","").replace('"','').strip() for s in single_writer_name]
    for s in single_writer_name:
        if s != '' and s not in names:
            names.append(s)
    
    # Directors
    if not pd.isna(row['Directors']):
        single_director_name = row['Directors'].split(',')
        single_director_name = [s.strip() for s in single_director_name]
        for s in single_director_name:
            if s != '' and s not in names:
                names.append(s)
    
names = list(set(names))
names.sort()
names.reverse()

names_copy = names

In [114]:
count = 0

for name in list(names):
    count += 1
    if count % 1000 == 0:
        print(count)
        
    if name not in names:
        continue
        
    names.remove(name)
    
    items = process.extract(name,filter(lambda n: n[0] == name[0], names),scorer=fuzz.ratio)
    filter_items = [i for i in items if i[1] >= 95 and i[1] != 100]
    
    if filter_items != []:
#         print(name)
#         print(filter_items)
        for f in set(filter_items):
#             print(f)
            if f in names:
#                 remove_values_from_list(names,f)
                names.remove(f)
            merged['Actors'].replace(regex=r"'" + r"{}".format(f[0]) + r"'", value="'" + name + "'" , inplace=True)            
            merged['Writers'].replace(regex=r"'" + r"{}".format(f[0]) + r"'", value="'" + name + "'" , inplace=True)            
            merged['Directors'].replace(regex= f[0], value=name , inplace=True)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000


In [117]:
merged.to_csv('../Data/movies_info_final.csv', index=False, sep=',')